In [12]:
import pandas as pd
import pandas_dedupe

## GETTING PERMISSION ERROR ON WINDOWS
from libs.feeds.postgres_feed import PostgresFeed
from libs.connectors.postgres_connector import PostgresConnector

source_host = 'localhost'
source_user = 'postgres'
source_passwd = 'postgres'
source_db = 'silobuster_testing'
source_port = 5432

source_conn = PostgresConnector(db=source_db, username=source_user, password=source_passwd, host=source_host, port=source_port)

manual_query = f"""
    SELECT
          id
        , name as o_name
        , description
        , url as o_url
        , address_1
        , address_2
        , city
        , state_province
        , postal_code
        , country
        , type
        , region
        , duplicate_id
        , duplicate_type
        , training_set
    FROM organizations_mangled
"""
pg_feed = PostgresFeed.from_manual(connector = source_conn, query = manual_query, column_definition=None, primary_key='id')

print(pg_feed.query)
print(pg_feed.df.head(5))

clean_df = pg_feed.df.replace('', None)

# The Levenshtein distance is used for fields identified as 'String'. However, it appears we 
# can't use "has missing" with Strings. Where fields may be empty, use type of "Test" instead.
df_final = pandas_dedupe.dedupe_dataframe(clean_df, [
    ('name', 'Text', 'has missing'),
    ('address_1', 'Text', 'has missing'), 
    ('o_url','Text','has missing'), 
    ('region', 'Text', 'has missing'),
    ('country', 'Text', 'has missing')
])

df_final.to_csv('deduplication_output_manual.csv')


    SELECT
          id
        , name as o_name
        , description
        , url as o_url
        , address_1
        , address_2
        , city
        , state_province
        , postal_code
        , country
        , type
        , region
        , duplicate_id
        , duplicate_type
        , training_set
    FROM organizations_mangled

   id                                             o_name  \
0   1                                    *required* text   
1   2                                    *required* text   
2   3  depressive and bipolar support alliance whatco...   
3   4  depressive and bipolar supkort allance whatom ...   
4   5           bellingham parks and recreation division   

                                         description  \
0                                    *required* text   
1                                    *required* text   
2  peer-directed national organization focusing o...   
3  peer-directed national organization focusing o...   
4  the pa

C:\Users\minim\AppData\Local\Temp\ipykernel_1976\4014435564.py:44: FutureWarning: The default value of regex will change from True to False in a future version.
  df_final = pandas_dedupe.dedupe_dataframe(clean_df, [


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\minim\\AppData\\Local\\Temp\\tmpakacgowd'